# Iris Neural Network

In [17]:
# Reference links:
#    Adapted from: https://github.com/emerging-technologies/keras-iris
#    Data set pulled from: https://gist.github.com/curran/a08a1080b88344b0c8a7

# imports and preliminaries
import csv
import numpy as np
import keras as kr

# Load the Iris dataset.
iris = list(csv.reader(open('iris-data-set.csv')))[1:]

Now that we have the data set loaded we can extrapolate the data we need into appropriate data sets in preparation for training and testing our Model

In [21]:
# The inputs are four floats: sepal length, sepal width, petal length, petal width.
inputs  = np.array(iris)[:,:4].astype(np.float)

In [19]:
# Outputs are initially individual strings: setosa, versicolor or virginica.
outputs = np.array(iris)[:,4]

# Convert the output strings to ints.
outputs_vals, outputs_ints = np.unique(outputs, return_inverse=True)

# Encode the category integers as binary categorical vairables.
outputs_cats = kr.utils.to_categorical(outputs_ints)

We can now randomly split the data into two sets: One for training and one for evaluation

In [26]:
# Split the input and output data sets into training and test subsets.
inds = np.random.permutation(len(inputs))
train_inds, test_inds = np.array_split(inds, 2)
inputs_train, outputs_train = inputs[train_inds], outputs_cats[train_inds]
inputs_test,  outputs_test  = inputs[test_inds],  outputs_cats[test_inds]

In [29]:
test_inds

array([ 20,  62, 143,  25,   7,  21,  80,   2,  31,  40, 118,  70,  84,
        86,  83, 111,  98,  28,  63, 148,  42,  41,  51,  11, 129,  95,
        22,  53, 144,  18,  99,  33,  43, 135, 130,  67,  37,   6, 136,
       117, 132, 116, 101, 115, 103, 109,   9,  77,  17, 121, 147,   5,
        66,  76,  52,  23,  26,   4,  39, 106,  58,  13, 137, 119,  87,
        10,   1,  16,  85,  56, 102, 122,  60,   8,  73])